In [1]:
import sys
sys.path.append('src')
import trie, utils, rlp

# Ex1

In [6]:
#create empty trie
state = trie.Trie('trie_db', trie.BLANK_ROOT)

#add key-value pair (010102, 'hello')
state.update('\x01\x01\x02', rlp.encode('hello'))

#check root hash
print 'root hash', state.root_hash.encode('hex')

k, v = state.root_node
#check node at the root
print 'root node:', [k, v]

#check the encoded key
print 'hp encoded key, in hex:', k.encode('hex')

root hash 46e7195cf3f2bb2f8bc6a4e5a04e42e48b056efb31e4170610d59a1da3a862d9
root node: [' \x01\x01\x02', '\x85hello']
hp encoded key, in hex: 20010102


# Ex2

In [101]:
#initialize trie
state = trie.Trie('triedb', trie.BLANK_ROOT)
state.update('\x01\x01\x02', rlp.encode(['hello']))
print 'root hash', state.root_hash.encode('hex')
k, v = state.root_node
print 'root node:', [k, v]
print 'hp encoded key, in hex:', k.encode('hex')

root hash 4a5b19d151e796482b08a1e020f1f7ef5ea7240c0171fd629598fee612892a7b
root node: [' \x01\x01\x02', '\xc6\x85hello']
hp encoded key, in hex: 20010102


In [102]:
state.update('\x01\x01\x02\x01', rlp.encode(['test1']))
state.update('\x01\x01\x02\x12', rlp.encode(['test2']))

print '---------AFTER UPDATE----------'
print 'root hash', state.root_hash.encode('hex')
k, v = state.root_node
print 'root node:', [k, v]
print 'hp encoded key, in hex:', k.encode('hex')

common_prefix_key, node_hash = state.root_node
print state._decode_to_node(node_hash)

---------AFTER UPDATE----------
root hash d317ba2ed839b01ea6be74401249a7112aed2643666927b734578f8f2334db28
root node: ['\x00\x01\x01\x02', '6tj4\xa9\xf2SJoxi;\xdd\xa0}H\x1a\xddM9\xb6\x8f0\xee\x1c\xfd+q(VY`']
hp encoded key, in hex: 00010102
[['1', '\xc6\x85test1'], ['2', '\xc6\x85test2'], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '\xc6\x85hello']


In [103]:

state.update('\x01\x01\x02\x25\xab\xcd', rlp.encode(['test1']))
state.update('\x01\x01\x02\x36\xef\x12', rlp.encode(['test2']))

print '---------AFTER UPDATE2----------'
print 'root hash', state.root_hash.encode('hex')
k, v = state.root_node
print 'root node:', [k, v]
print 'hp encoded key, in hex:', k.encode('hex')

common_prefix_key, node_hash = state.root_node
print state._decode_to_node(node_hash)

---------AFTER UPDATE2----------
root hash 3f92ddf32437463622180645b71099cfc708568da17a9346bb986b9f153b251d
root node: ['\x00\x01\x01\x02', '\x92\xbbF\xe8;\tr\x00tk\x93V\x9be\xaa\xcc\\\x16u\x1d\x93;(\xe5U5)\xe8G{\xd4_']
hp encoded key, in hex: 00010102
[['1', '\xc6\x85test1'], ['2', '\xc6\x85test2'], ['5\xab\xcd', '\xc6\x85test1'], ['6\xef\x12', '\xc6\x85test2'], '', '', '', '', '', '', '', '', '', '', '', '', '\xc6\x85hello']


# Ex3

In [105]:
#initialize trie
state = trie.Trie('triedb', trie.BLANK_ROOT)
state.update('\x01\x01\x02', rlp.encode(['hello']))
print 'root hash', state.root_hash.encode('hex')
k, v = state.root_node
print 'root node:', [k, v]
print 'hp encoded key, in hex:', k.encode('hex')

root hash 4a5b19d151e796482b08a1e020f1f7ef5ea7240c0171fd629598fee612892a7b
root node: [' \x01\x01\x02', '\xc6\x85hello']
hp encoded key, in hex: 20010102


In [106]:
state.update('\x01\x01\x12', rlp.encode(['test1']))
state.update('\x01\x01\x13', rlp.encode(['test2']))

print '---------------------'
print 'root hash', state.root_hash.encode('hex')
k, v = state.root_node
print 'root node:', [k, v]
print 'hp encoded key, in hex:', k.encode('hex')

common_prefix_key, node_hash = state.root_node
print state._decode_to_node(node_hash)

---------------------
root hash 53733ab375e80275bd427b3f11ab15635c92e530d9583d917cc75b50410d829f
root node: ['\x00\x01\x01', '\xe4\xc6\xaf\xd81\xd1\xbd_\xd5\x8a\x03\xea~Vh5dr2\x17b\xf6\xd7\x86\x15e\xd1kC*\xa0T']
hp encoded key, in hex: 000101
[['2', '\xc6\x85hello'], '\xee\xfb\xe07\x18\xa5=\xaa\xfc\x12\xab\xc5HKWl$\xbbxI\x15\x8a\xb9\x80\xce\x15\xbbebL\x1b\xa6', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']


In [109]:
next_node_hash = state._decode_to_node(state.root_node[1])[1]
print next_node_hash.encode('hex')
print state._decode_to_node(next_node_hash)

eefbe03718a53daafc12abc5484b576c24bb7849158ab980ce15bb65624c1ba6
['', '', [' ', '\xc6\x85test1'], [' ', '\xc6\x85test2'], '', '', '', '', '', '', '', '', '', '', '', '', '']


# Ex4

In [118]:
#initialize trie
state = trie.Trie('triedb', trie.BLANK_ROOT)
state.update('\x01\x01\x02', rlp.encode(['hello']))
print 'root hash', state.root_hash.encode('hex')
k, v = state.root_node
print 'root node:', [k, v]
print 'hp encoded key, in hex:', k.encode('hex')

root hash 4a5b19d151e796482b08a1e020f1f7ef5ea7240c0171fd629598fee612892a7b
root node: [' \x01\x01\x02', '\xc6\x85hello']
hp encoded key, in hex: 20010102


In [3]:
state.update('\x01\x01\x03', rlp.encode(['test1']))
state.update('\x01\x01\x05\x11\x22\x12', rlp.encode(['test2']))
print '---------------------'
print 'root hash', state.root_hash.encode('hex')
k, v = state.root_node
print 'root node:', [k, v]
print 'hp encoded key, in hex:', k.encode('hex')
common_prefix_key, node_hash = state.root_node
print state._decode_to_node(node_hash)

---------------------
root hash 9f9292dff56eefb7dc2ea372f82908dfcfabe1deee074fa0fd02c71de8bd2384
root node: ['\x10\x10\x10', '\x95B\xe2@\xe3z\n\xe2eI>Ic\t>t\xe9Yt\x0e+\xecTF\xe0\xb5\x1d|kG\xfd\xf2']
hp encoded key, in hex: 101010
['', '', [' ', '\xc6\x85hello'], [' ', '\xc6\x85test1'], '', [' \x11"\x12', '\xc6\x85test2'], '', '', '', '', '', '', '', '', '', '', '']


# Ex5

In [111]:
#initialize trie
state = trie.Trie('triedb', trie.BLANK_ROOT)
state.update('\x01\x01\x02', rlp.encode(['hello']))
print 'root hash', state.root_hash.encode('hex')
k, v = state.root_node
print 'root node:', [k, v]
print 'hp encoded key, in hex:', k.encode('hex')

root hash 4a5b19d151e796482b08a1e020f1f7ef5ea7240c0171fd629598fee612892a7b
root node: [' \x01\x01\x02', '\xc6\x85hello']
hp encoded key, in hex: 20010102


In [112]:
state.update('\x01\x01\x12\x11\x22', rlp.encode(['test1']))
state.update('\x01\x01\x13\x12\x13', rlp.encode(['test2']))

print '---------------------'
print 'root hash', state.root_hash.encode('hex')
k, v = state.root_node
print 'root node:', [k, v]
print 'hp encoded key, in hex:', k.encode('hex')

common_prefix_key, node_hash = state.root_node
print state._decode_to_node(node_hash)

---------------------
root hash f09fca588bb29c99b52d36a94540613e73f76d8b8aa510a6a5dd3522cf4b742a
root node: ['\x00\x01\x01', '\xa1\xd9\xe5\xe6\xf0/\x05\x88\xdc\xd6IC\x8d\xcf\x8c\x87\xc6\xfa\xc89\xa6\x85d]!E\xc3\xfey\x08UB']
hp encoded key, in hex: 000101
[['2', '\xc6\x85hello'], '\x01\x9fb~\xe2\xd7\xf1\n\x07\x055:_\xe4\xd5\x1b{\x19\xf6\xc8\x0c7\x07\xac\x86@\xf9\x05s\x11\xd0z', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']


In [117]:
next_node_hash = state._decode_to_node(state.root_node[1])[1]
print next_node_hash.encode('hex')
print state._decode_to_node(next_node_hash)

019f627ee2d7f10a0705353a5fe4d51b7b19f6c80c3707ac8640f9057311d07a
['', '', [' \x11"', '\xc6\x85test1'], [' \x12\x13', '\xc6\x85test2'], '', '', '', '', '', '', '', '', '', '', '', '', '']


# Ex7

follow this example https://ethereum.org/en/developers/docs/data-structures-and-encoding/patricia-merkle-trie/

In [15]:
def str_to_hex(s):
    return ''.join(['\\x{:02x}'.format(ord(c)) for c in s])

first_leaf = ('do', 'verb')
second_leaf = ('dog', 'puppy')
third_leaf = ('doge', 'coin')
forth_leaf = ('horse', 'stallion')

'646f'

In [14]:

# Test
input_str = "do"
print(str_to_hex(input_str))


\x64\x6f


# Ex8

In [31]:
#initialize trie
state = trie.Trie('triedb', trie.BLANK_ROOT)
state.update('\x01\x01\x02', rlp.encode(['hello']))
print 'root hash', state.root_hash.encode('hex')
k, v = state.root_node
print 'root node:', [k, v]
print 'hp encoded key, in hex:', k.encode('hex')

root hash 4a5b19d151e796482b08a1e020f1f7ef5ea7240c0171fd629598fee612892a7b
root node: [' \x01\x01\x02', '\xc6\x85hello']
hp encoded key, in hex: 20010102


In [32]:
state.update('\x01\x01\x12', rlp.encode(['test1']))
state.update('\x01\x01\x13', rlp.encode(['test2']))

print '---------------------'
print 'root hash', state.root_hash.encode('hex')
k, v = state.root_node
print 'root node:', [k, v]
print 'hp encoded key, in hex:', k.encode('hex')

common_prefix_key, node_hash = state.root_node
print state._decode_to_node(node_hash)
print state._get_node_type(state._decode_to_node(node_hash))

---------------------
root hash 53733ab375e80275bd427b3f11ab15635c92e530d9583d917cc75b50410d829f
root node: ['\x00\x01\x01', '\xe4\xc6\xaf\xd81\xd1\xbd_\xd5\x8a\x03\xea~Vh5dr2\x17b\xf6\xd7\x86\x15e\xd1kC*\xa0T']
hp encoded key, in hex: 000101
[['2', '\xc6\x85hello'], '\xee\xfb\xe07\x18\xa5=\xaa\xfc\x12\xab\xc5HKWl$\xbbxI\x15\x8a\xb9\x80\xce\x15\xbbebL\x1b\xa6', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
3


In [33]:
trie.NODE_TYPE_BRANCH

3

In [34]:
trie.NODE_TYPE_LEAF

1

In [35]:
trie.NODE_TYPE_EXTENSION

2